<a href="https://colab.research.google.com/gist/Mndvv/e13deca40c6f8690a16802ab7b5fd3a7/smartstudent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from IPython.display import clear_output as clearoutput, Image, display
from tabulate import tabulate
from datetime import datetime
from zipfile import ZipFile
import sys, math, pandas as pd, requests as req, random, string, re
import openpyxl, time

try:
  import colorama as cr, qrcode
except ImportError:
  !pip install colorama qrcode[pil]
  import colorama as cr, qrcode

user_data = {
    "nama": None,
    "kelamin": None,
    "umur": None,
    "berat_badan": None,
    "tinggi_badan": None,
    "riwayat_diagnosis": []
}

def keluar_program():
    return print(f"-> {cr.Fore.GREEN}Anda keluar dari aplikasi, terimakasih telah menggunakan!")

def clear_output():
    clearoutput()
    time.sleep(0.2)

# Dokterku Section -------------------------------------------------------------
def klasifikasikan_imt(imt, klasifikasi="teks"):
    match imt:
        case imt if imt < 16.5:  # Gizi kurang (underweight)
            return cr.Back.RED if klasifikasi == "warna" else (
                "Hasil IMT menunjukkan nilai gizi Anda sangat kurang, berada di bawah angka yang seharusnya (underweight). "
                "\nAnda perlu meningkatkan asupan kalori dan gizi agar berat badan Anda dapat mencapai kondisi yang sehat. "
                "\nSaran: Konsultasikan dengan ahli gizi atau dokter untuk mendapatkan program diet yang sesuai, "
                "\ndan perbanyak konsumsi makanan bergizi tinggi seperti protein, lemak sehat, serta karbohidrat kompleks."
            )
        case imt if 16.5 <= imt < 18.5:  # Gizi kurang, tapi sedikit lebih baik
            return cr.Back.ORANGE if klasifikasi == "warna" else (
                "Hasil IMT menunjukkan bahwa Anda berada dalam kategori gizi kurang, meskipun sedikit lebih baik dibandingkan kondisi sangat kurus. "
                "\nAnda masih membutuhkan perbaikan dalam asupan makanan untuk mencapai berat badan yang sehat. "
                "\nSaran: Tingkatkan konsumsi makanan yang lebih bergizi, dengan fokus pada peningkatan protein dan kalori dalam diet Anda."
            )
        case imt if 18.5 <= imt <= 22.9:  # IMT normal
            return cr.Back.GREEN if klasifikasi == "warna" else (
                "Hasil IMT menunjukkan bahwa nilai gizi Anda berada dalam kategori normal. "
                "\nAnda memiliki berat badan ideal sesuai dengan tinggi badan. "
                "\nSaran: Terus pertahankan pola makan sehat dan gaya hidup aktif untuk menjaga kesehatan tubuh."
            )
        case imt if 23 <= imt < 25:  # Risiko gizi lebih
            return cr.Back.ORANGE if klasifikasi == "warna" else (
                "Hasil IMT menunjukkan bahwa Anda berada dalam kategori risiko gizi lebih (pre-obesitas). "
                "\nMeskipun Anda tidak tergolong obesitas, namun perlu perhatian agar berat badan Anda tidak semakin bertambah. "
                "\nSaran: Mulailah menjaga pola makan sehat, mengurangi asupan makanan berkalori tinggi, dan rutin berolahraga untuk menghindari peningkatan berat badan yang tidak sehat."
            )
        case _:  # Obesitas
            return cr.Back.RED if klasifikasi == "warna" else (
                "Hasil IMT menunjukkan bahwa Anda memiliki kelebihan berat badan (obesitas). "
                "\nKondisi ini dapat meningkatkan risiko berbagai penyakit seperti diabetes, hipertensi, dan penyakit jantung. "
                "\nSaran: Segera konsultasikan dengan dokter atau ahli gizi untuk menyusun program diet yang sehat dan aman. "
                "\nLakukan olahraga secara teratur dan hindari makanan yang tinggi lemak dan kalori."
            )

def kalkulator_imt():
      clear_output()
      print("\t","="*30, f"\n\t{cr.Back.BLUE}Kalkulator Indeks Massa Tubuh{cr.Back.RESET}\n\t", "="*30,"\nisi pertanyaan dibawah untuk mendapatkan hasil:\n")
      while True:
        try:
          if user_data["kelamin"] is None:
            k = input("Anda laki-laki atau perempuan? (l/p): ")
            user_data["kelamin"] = (1 if k =="l" else 0 if k == "p" else None)
          elif user_data["umur"] is None:
            user_data["umur"] = int(input("Masukan umur anda (dalam tahun): "))
          elif user_data["berat_badan"] is None:
            user_data["berat_badan"] = int(input("Masukan berat badan anda (kg): "))
          elif user_data["tinggi_badan"] is None:
            user_data["tinggi_badan"] = int(input("Masukan tinggi badan anda (cm): "))
          else:
            break
        except ValueError:
          print(f"{cr.Fore.RED}Anda tidak benar memasukan data.{cr.Fore.RESET}")

      tinggi_m = user_data["tinggi_badan"] / 100
      imt = user_data["berat_badan"] / (tinggi_m**2)
      imt_result = klasifikasikan_imt(imt)
      print(f"\n{cr.Fore.GREEN} -> Skor Indeks Massa Tubuh Anda: {cr.Fore.RESET}{klasifikasikan_imt(imt, 'warna')}{round(imt, 2)}{cr.Fore.RESET+cr.Back.RESET}")
      print(f" {imt_result}")
      input("Tekan enter untuk kembali...")
      dokterku()

def cek_stress():
    clear_output()
    print("\t","="*30, f"\n\t{cr.Back.RED}Cek Tingkatan Stress{cr.Back.RESET}"+("-"*5)+">\n\t", "="*30,"\nisi kuisioner dibawah untuk mendapatkan hasil:\n")
    skor_total = 0
    pilihan_diagnosa = []
    poin_diagnosa = {
        "Asma": 4,
        "Diabetes": 5,
        "Penyakit jantung": 7,
        "Obesitas": 6,
        "Lainnya/mengganggu aktivitas": 7,
        "Lainnya/tidak menganggu aktivitas": 5
    }

    pertanyaan_stress = [
        {
            "teks": "Seberapa sering Anda merasa sulit untuk rileks?",
            "jawaban": ["Tidak pernah", "Jarang", "Kadang-kadang", "Sering", "Sangat sering"],
            "nilai": [0, 1, 2, 3, 4]
        },
        {
            "teks": "Seberapa sering Anda mudah marah atau tersinggung?",
            "jawaban": ["Tidak pernah", "Jarang", "Kadang-kadang", "Sering", "Sangat sering"],
            "nilai": [0, 1, 2, 3, 4]
        },
        {
            "teks": "Seberapa sering Anda kesulitan tidur karena suatu masalah?",
            "jawaban": ["Tidak pernah", "Jarang", "Kadang-kadang", "Sering", "Sangat sering"],
            "nilai": [0, 1, 2, 3, 4]
        },
        {
            "teks": "Seberapa sering Anda merasa cemas atau khawatir berlebihan?",
            "jawaban": ["Tidak pernah", "Jarang", "Kadang-kadang", "Sering", "Sangat sering"],
            "nilai": [0, 1, 2, 3, 4]
        },
        {
            "teks": "Seberapa sering Anda mengalami sakit kepala karena hal-hal disekitar anda atau suatu masalah?",
            "jawaban": ["Tidak pernah", "Jarang", "Kadang-kadang", "Sering", "Sangat sering"],
            "nilai": [0, 1, 2, 3, 4]
        }
    ]

    while True:
        try:
            if user_data["kelamin"] is None:
                k = input("Anda laki-laki atau perempuan? (l/p): ").lower()
                user_data["kelamin"] = (1 if k == "l" else 0 if k == "p" else None)
            elif user_data["umur"] is None:
                user_data["umur"] = int(input("Masukan umur anda (dalam tahun, cth: 15): "))
            elif not pilihan_diagnosa:
                print("\nApakah anda memiliki riwayat penyakit berikut?")
                for i, name in enumerate(poin_diagnosa.keys(), start=1):
                    print(f" {i}. {name}")
                pilihan = input("\nPilih nomor diagnosis yang relevan dengan anda:\n(pisah dengan koma jika lebih dari 1, kosongkan jika tidak ada): ").strip()

                if pilihan:
                    splitted_input = [int(num.strip()) for num in pilihan.split(",") if num.strip().isdigit()]
                    for num in splitted_input:
                        if 1 <= num <= len(poin_diagnosa):
                            penyakit = list(poin_diagnosa.keys())[num - 1]
                            if penyakit not in pilihan_diagnosa:
                              skor_total += poin_diagnosa[penyakit]
                              pilihan_diagnosa.append(penyakit)
                        else:
                            print(f"Error: Nomor {num} tidak valid.")
                            pilihan_diagnosa=[]
                            continue
                if pilihan_diagnosa:
                  user_data["riwayat_diagnosis"] = pilihan_diagnosa
                  break
            else:
                break
        except ValueError:
            print(f"{cr.Fore.RED}Anda tidak benar memasukan data dengan benar.{cr.Fore.RESET}")

    print(f"\n{cr.Fore.YELLOW}Silakan jawab pertanyaan berikut untuk evaluasi stress:{cr.Fore.RESET}")
    for i, pertanyaan in enumerate(pertanyaan_stress, 1):
        print(f"\n{cr.Fore.GREEN}Pertanyaan {i}: {pertanyaan['teks']}{cr.Fore.RESET}")
        for j, jawaban in enumerate(pertanyaan["jawaban"], 1):
            print(f" {j}. {jawaban}")

        while True:
            try:
                jawab = int(input(f"Pilih jawaban (1-{len(pertanyaan['jawaban'])}): "))
                if 1 <= jawab <= len(pertanyaan["jawaban"]):
                    skor_total += pertanyaan["nilai"][jawab-1]
                    break
                else:
                    print(f"{cr.Fore.RED}Pilihan tidak valid. Silakan pilih antara 1-{len(pertanyaan['jawaban'])}.{cr.Fore.RESET}")
            except ValueError:
                print(f"{cr.Fore.RED}Input harus berupa angka.{cr.Fore.RESET}")

    clear_output()
    print("\t","="*30, f"\n\t{cr.Back.RED}Hasil Evaluasi Stress{cr.Back.RESET}\n\t", "="*30)

    if skor_total <= 5:
        tingkat_stress = f"{cr.Back.GREEN}Rendah{cr.Back.RESET}"
        deskripsi = "Tingkat stress Anda normal dan tidak mengkhawatirkan."
        saran = "Tetap jaga pola hidup sehat dan lakukan aktivitas yang menyenangkan untuk menjaga keseimbangan mental."
    elif skor_total <= 10:
        tingkat_stress = f"{cr.Back.YELLOW}Sedang{cr.Back.RESET}"
        deskripsi = "Anda mengalami tingkat stress sedang yang umum dialami pelajar."
        saran = "Pertimbangkan untuk lebih banyak istirahat, olahraga ringan, dan teknik relaksasi sederhana seperti pernapasan dalam."
    elif skor_total <= 15:
        tingkat_stress = f"{cr.Back.RED}Tinggi{cr.Back.RESET}"
        deskripsi = "Anda mengalami tingkat stress yang cukup tinggi dan perlu penanganan."
        saran = "Penting untuk berbicara dengan konselor sekolah atau profesional kesehatan mental. Coba kurangi beban tugas dan prioritaskan waktu istirahat."
    else:
        tingkat_stress = f"{cr.Back.RED+cr.Fore.WHITE}Sangat Tinggi{cr.Back.RESET+cr.Fore.RESET}"
        deskripsi = "Tingkat stress Anda sangat tinggi dan dapat berdampak negatif pada kesehatan fisik maupun mental."
        saran = "Segera konsultasikan dengan profesional kesehatan mental. Stress berkepanjangan dapat mempengaruhi kesehatan dan prestasi akademik Anda."

    if pilihan_diagnosa:
        saran += f"\n\nPerhatian khusus: Dengan riwayat {', '.join(pilihan_diagnosa)}, penting untuk lebih menjaga tingkat stress Anda, dan jangan terlalu banyak pikiran."

    print(f"\n{cr.Fore.YELLOW}Nama: {cr.Fore.RESET}{user_data['nama']}")
    print(f"{cr.Fore.YELLOW}Umur: {cr.Fore.RESET}{user_data['umur']} tahun")
    print(f"{cr.Fore.YELLOW}Jenis Kelamin: {cr.Fore.RESET}{('Laki-laki' if user_data['kelamin'] else 'Perempuan')}")
    if pilihan_diagnosa:
        print(f"{cr.Fore.YELLOW}Riwayat Diagnosis: {cr.Fore.RESET}{', '.join(pilihan_diagnosa)}")

    print(f"\n{cr.Fore.GREEN}Skor Total: {cr.Fore.RESET}{skor_total}")
    print(f"{cr.Fore.GREEN}Tingkat Stress: {cr.Fore.RESET}{tingkat_stress}")
    print(f"\n{cr.Fore.BLUE}Deskripsi:{cr.Fore.RESET} {deskripsi}")
    print(f"{cr.Fore.BLUE}Saran:{cr.Fore.RESET} {saran}")

    input("\nTekan enter untuk kembali...")
    dokterku()

def cek_kecemasan():
    print("\t","="*30, f"\n\t{cr.Back.GREEN}Tes Gangguan Kecemasan{cr.Back.RESET}"+("-"*5)+">\n\t", "="*30,"\nTes gangguan kecemasan dengan tes Generalized Anxiety Disorder-7\n")
    skor_total = 0

    pertanyaan_kecemasan = [
        {
            "teks": "Merasa gugup, cemas, atau gelisah?",
            "jawaban": ["Tidak pernah", "Beberapa Hari", "Sebagian Besar Hari", "Hampir Setiap Hari"],
            "nilai": [0, 1, 2, 3]
        },
        {
            "teks": "Tidak dapat menghentikan atau mengontrol kekhawatiran?",
            "jawaban": ["Tidak pernah", "Beberapa Hari", "Sebagian Besar Hari", "Hampir Setiap Hari"],
            "nilai": [0, 1, 2, 3]
        },
        {
            "teks": "Terlalu banyak mengkhawatirkan berbagai hal?",
            "jawaban": ["Tidak pernah", "Beberapa Hari", "Sebagian Besar Hari", "Hampir Setiap Hari"],
            "nilai": [0, 1, 2, 3]
        },
        {
            "teks": "Sulit merasa santai?",
            "jawaban": ["Tidak pernah", "Beberapa Hari", "Sebagian Besar Hari", "Hampir Setiap Hari"],
            "nilai": [0, 1, 2, 3]
        },
        {
            "teks": "Merasa kurang istirahat hingga sulit untuk diam?",
            "jawaban": ["Tidak pernah", "Beberapa Hari", "Sebagian Besar Hari", "Hampir Setiap Hari"],
            "nilai": [0, 1, 2, 3]
        },
        {
            "teks": "Mudah kesal atau marah?",
            "jawaban": ["Tidak pernah", "Beberapa Hari", "Sebagian Besar Hari", "Hampir Setiap Hari"],
            "nilai": [0, 1, 2, 3]
        },
        {
            "teks": "Merasa takut hal buruk akan terjadi?",
            "jawaban": ["Tidak pernah", "Beberapa Hari", "Sebagian Besar Hari", "Hampir Setiap Hari"],
            "nilai": [0, 1, 2, 3]
        }
    ]


    print(f"\n{cr.Fore.YELLOW}Dalam 2 minggu terakhir,\nSeberapa sering kamu merasa terganggu oleh hal berikut...{cr.Fore.RESET}")
    for i, pertanyaan in enumerate(pertanyaan_kecemasan, 1):
        print(f"\n{cr.Fore.GREEN}Pertanyaan {i}: {pertanyaan['teks']}{cr.Fore.RESET}")
        for j, jawaban in enumerate(pertanyaan["jawaban"], 1):
            print(f" {j}. {jawaban}")

        while True:
            try:
                jawab = int(input(f"Pilih jawaban (1-{len(pertanyaan['jawaban'])}): "))
                if 1 <= jawab <= len(pertanyaan["jawaban"]):
                    skor_total += pertanyaan["nilai"][jawab-1]
                    break
                else:
                    print(f"{cr.Fore.RED}Pilihan tidak valid. Silakan pilih antara 1-{len(pertanyaan['jawaban'])}.{cr.Fore.RESET}")
            except ValueError:
                print(f"{cr.Fore.RED}Input harus berupa angka.{cr.Fore.RESET}")

    clear_output()
    print("\t","="*30, f"\n\t{cr.Back.BLUE}Hasil Evaluasi Generalized Anxiety Disorder-7{cr.Back.RESET}\n\t", "="*30)

    if skor_total <= 5:
        tingkat_kecemasan = f"{cr.Back.GREEN}Rendah{cr.Back.RESET}"
        deskripsi = "Hore! Skor kamu menunjukkan tidak ada gejala gangguan kecemasan.."
        saran = "Jika kamu merasa ada gejala yang berkembang atau perlu ruang aman untuk curhat, konsultasi pada ahli."
    elif skor_total < 10:
        tingkat_kecemasan = f"{cr.Back.YELLOW}Sedang{cr.Back.RESET}"
        deskripsi = "Skor kamu menandakan gangguan kecemasan ringan."
        saran = "Biasanya kamu masih dapat mengontrol kecemasanmu, namun kamu merasakan keresahan yang tidak dapat dijelaskan. Konsultasi dengan ahli kesehatan mental untuk mencari tahu penyebab dan solusinya."
    elif skor_total < 15:
        tingkat_kecemasan = f"{cr.Back.RED}Tinggi{cr.Back.RESET}"
        deskripsi = "Skor kamu menandakan gangguan kecemasan Tinggi."
        saran = "Kamu sering merasa gelisah dan kesulitan mengontrol kecemasanmu. Gejala fisik seperti gangguan tidur dan kesulitan berfokus mungkin lebih terasa, dan meningkat pada saat stres. Dapatkan bantuan ahli untuk memahami pikiran, perasaan, dan gejala yang kamu alami."
    else:
        tingkat_kecemasan = f"{cr.Back.RED+cr.Fore.WHITE}Berat{cr.Back.RESET+cr.Fore.RESET}"
        deskripsi = "Skor kamu menandakan gangguan kecemasan berat."
        saran = "Segera dapatkan pemeriksaan dari ahli kesehatan mental. Jika tidak diatasi, gangguan kecemasan berat dapat memengaruhi kemampuanmu melakukan aktivitas sehari-hari, mengambil keputusan, & merawat diri.."

    print(f"\n{cr.Fore.YELLOW}Nama: {cr.Fore.RESET}{user_data['nama']}")
    if(user_data['umur']):
      print(f"{cr.Fore.YELLOW}Umur: {cr.Fore.RESET}{user_data['umur']} tahun")
    if(user_data['kelamin'] is not None):
      print(f"{cr.Fore.YELLOW}Jenis Kelamin: {cr.Fore.RESET}{('Laki-laki' if user_data['kelamin'] else 'Perempuan')}")

    print(f"\n{cr.Fore.GREEN}Skor Total: {cr.Fore.WHITE}{skor_total}")
    print(f"{cr.Fore.GREEN}Tingkat Stress: {cr.Fore.RESET}{tingkat_kecemasan}")
    print(f"\n{cr.Fore.BLUE}Deskripsi:{cr.Fore.RESET} {deskripsi}")
    print(f"{cr.Fore.BLUE}Saran:{cr.Fore.RESET} {saran}")

    input("\nTekan enter untuk kembali...")
    dokterku()

def dokterku():
    dokterkusapa=f"👨‍⚕️{cr.Fore.BLUE}\x1b[3mDokterku disini!\x1b[0m Pilih pilihan dibawah ya."
    dokterkuMaps = {
        f"🏋️{cr.Fore.GREEN}Kalkulator Berat Badan Ideal ({cr.Fore.BLUE}IMT{cr.Fore.RESET})" : kalkulator_imt,
        f"‼️{cr.Fore.RED}Cek Stress{cr.Fore.RESET}" : cek_stress,
        f"😰{cr.Fore.RED}Cek Gangguan Kecemasan{cr.Fore.RESET}" : cek_kecemasan,
        f"⬅️{cr.Fore.LIGHTBLACK_EX}Kembali{cr.Fore.RESET}" : lambda: menu_awal()
    }
    pilihanTersedia=list(dokterkuMaps)
    listPilihan(dokterkusapa,dokterkuMaps)

    while True:
      try:
        pilihan = int(input(f"Pilih aplikasi (1-{len(dokterkuMaps)}): "))
        if 1 <= pilihan <= (len(pilihanTersedia)):
          dokterkuMaps[pilihanTersedia[pilihan-1]]()
          break
        else:
          listPilihan(dokterkusapa,dokterkuMaps,"Pilihan tidak ada!")
      except ValueError:
        listPilihan(dokterkusapa,dokterkuMaps,"Input tidak valid! Harus berupa angka sesuai list.")
      except KeyboardInterrupt:
        print(f"\n👨‍⚕️{cr.Fore.BLUE}Dokterku pamit ya...")
        break
# Dokterku section END ---------------------------------------------------------

# Sabela section ---------------------------------------------------------------
def musik_relaksasi():
    print(f"\n{cr.Fore.BLUE}Klik disini untuk playlist musik relaksasi{cr.Fore.GREEN}->{cr.Fore.RESET} https://www.youtube.com/watch?v=mPZkdNFkNps&list=PLrP2GOuYiQv4MfCHNij1kEfhL9KgbtjJa&pp=gAQB")
    input("\nTekan enter untuk kembali...")
    sahabat_belajar()


def qr_creator():
    clear_output()
    data = input("Masukan link/data yang ingin dijadikan kode QR: ")
    if(data):
        qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=7, border=2)

        qr.add_data(data)
        qr.make(fit=True)

        # Membuat gambar dan menyimpan ke file.
        img = qr.make_image(fill_color="black", back_color="white")
        img.save("qr_output.png")

        display(Image("qr_output.png"))
    else:
      print("Anda tidak memasukan data apapun.") # too lazy to create another while loop

    input("\nTekan enter untuk kembali...")
    sahabat_belajar()


def saran_ai():
    clear_output()
    saran = f"""{cr.Back.CYAN} SARAN AI UNTUK PELAJAR {cr.Back.RESET}
      -> Riset & Bahan Studi:
        - https://gemini.google.com/app (DeepSearch)
        - https://chatgpt.com/g/g-kZ0eYXlJe-scholar-gpt
        - https://chatgpt.com (DeepSearch)
        - https://grok.com/ (DeeperResearch)
      -> Pembelajaran
        - https://chatgpt.com
        - https://grok.com
        - https://gemini.google.com/app
      -> Visualisasi Data
        - https://claude.ai/new
        - https://chat.deepseek.com/
      -> Dokumen
        - https://www.notion.com/product/ai
        - https://chatgpt.com
        - https://quillbot.com/
      Note: Cuma untuk alat & utilitas ya!, jangan bergantung sama AI, nanti dungu.
    """
    print(saran)
    input("\nTekan enter untuk kembali...")
    sahabat_belajar()


def sahabat_belajar():
    sabelamenyapa=f"👩‍🦰{cr.Fore.BLUE}\x1b[3mSabela(Sahabat Belajar) disini, ada apa hari ini?\x1b[0m Pilih pilihan dibawah ya."
    sabelaMaps = {
        f"🎶{cr.Fore.BLUE}Musik Relaksasi" : musik_relaksasi,
        f"📑{cr.Fore.GREEN}Pembuat Kode QR{cr.Fore.RESET}" : qr_creator,
        f"🤖{cr.Fore.CYAN}Saran AI Belajar{cr.Fore.RESET}" : saran_ai,
        f"⬅️{cr.Fore.LIGHTBLACK_EX}Kembali{cr.Fore.RESET}" : lambda: menu_awal()
    }
    pilihanTersedia=list(sabelaMaps)
    listPilihan(sabelamenyapa,sabelaMaps)

    while True:
      try:
        pilihan = int(input(f"Pilih aplikasi (1-{len(sabelaMaps)}): "))
        if 1 <= pilihan <= (len(pilihanTersedia)):
          sabelaMaps[pilihanTersedia[pilihan-1]]()
          break
        else:
          listPilihan(sabelamenyapa,sabelaMaps,"Pilihan tidak ada!")
      except ValueError:
        listPilihan(sabelamenyapa,sabelaMaps,"Input tidak valid! Harus berupa angka sesuai list.")
      except KeyboardInterrupt:
        print(f"\n😒{cr.Fore.BLUE}Okedeh, Sabela pamit ya...")
        break
# Sabela section END -----------------------------------------------------------

def kalkulator_pembayaran():
    clear_output()
    print(f"{cr.Fore.YELLOW}\t-> Kalkulator Pembayaran{cr.Fore.RESET}")
    spp = 0
    biaya_total = {}
    current_month = datetime.now().month
    month_names = ["Januari", "Februari", "Maret", "April", "Mei", "Juni",
                  "Juli", "Agustus", "September", "Oktober", "November", "Desember"]
    while True:
      try:
        if not spp:
          spp = int(input("Berapa biaya SPP Bulananmu?: Rp."))
        elif not biaya_total:
          decision=input(f"Apakah anda punya biaya bulanan atau tunggakan lain? (y/n): ").strip().lower()
          if decision == "y":
            biaya_lain_n = input("Masukan nama tunggakanmu yang lain, pisah dengan koma jika ada banyak: ").strip()
            biaya_total = {item.strip(): {"jumlah": None, "bulanan": False} for item in biaya_lain_n.split(",")}
            for item in biaya_total:
              while True:
                  try:
                      jumlah_bayaran = int(input(f"Masukan jumlah bayaran untuk {item}: Rp."))
                      bulanan_input = input(f"Apakah {item} adalah tunggakan bulanan? (y/n): ").strip().lower()
                      bulanan = bulanan_input == "y"

                      # Jika bukan bulanan, minta input bulan
                      bulan_input = None
                      if not bulanan:
                          while True:
                              bulan_input = input(f"Kapan {item} mau dibayar? (cth: November): ").strip().title()

                              # Validasi 1: Cek bulan ada di list
                              if bulan_input not in month_names:
                                  print(f"{cr.Fore.RED}❌ Bulan tidak valid. Gunakan format seperti 'November'.{cr.Fore.RESET}")
                                  continue

                              # Validasi 2: Cek bulan belum lewat
                              selected_month_idx = month_names.index(bulan_input) + 1
                              if selected_month_idx < current_month:
                                  print(f"{cr.Fore.RED}❌ Tidak bisa memilih bulan yang sudah lewat!{cr.Fore.RESET}")
                              else:
                                  break  # Keluar loop jika valid

                      # Simpan data
                      biaya_total[item] = {
                          "jumlah": jumlah_bayaran,
                          "bulanan": bulanan,
                          "dibayar": bulan_input if not bulanan else None
                      }
                      break  # Keluar dari loop utama

                  except ValueError:
                      print(f"{cr.Fore.RED}❗Input angka tidak valid!{cr.Fore.RESET}")
            if spp:
              biaya_total["SPP Bulanan"]={"jumlah":spp,"bulanan":True, "dibayar": None}
          elif decision == "n":
            if spp:
              biaya_total["SPP Bulanan"]={"jumlah":spp,"bulanan":True, "dibayar": None}
            break
          else:
            continue
        else:
          break
      except ValueError:
        print("Input tidak sesuai, harus angka.")
        continue
      except KeyboardInterrupt:
        print("Ok, Dadah! 😉")
        break

    clear_output()
    print(f"{cr.Fore.YELLOW}-> Masukan anda{cr.Fore.RESET}")
    df = pd.DataFrame.from_dict(biaya_total, orient='index')
    df.index.name = "Nama Tunggakan"
    df.reset_index(inplace=True)

    df["Tipe"] = df["bulanan"].apply(lambda x: "Bulanan" if x else "Sekali")
    df["Jumlah"] = df["jumlah"].apply(lambda x: f"Rp{x:,}".replace(",", "."))
    df["Dibayarkan"] = df["dibayar"]
    df = df.drop(columns=["jumlah", "bulanan", "dibayar"])

    # hitung data tunggakan
    monthly_data = []
    monthly_data_clean = []
    total_bulanan = sum(v['jumlah'] for v in biaya_total.values() if v['bulanan'])
    biaya_sekali_perbulan = {
        month_names.index(v['dibayar']) + 1: v['jumlah']
        for v in biaya_total.values()
        if not v['bulanan'] and v['dibayar']
    }


    for month_idx, bulan in enumerate(month_names, start=1):
        payment = total_bulanan
        # Tambahkan biaya sekali jika ada di bulan ini
        payment += biaya_sekali_perbulan.get(month_idx, 0)

        monthly_data_clean.append([bulan, f"Rp{payment:,}".replace(",", ".")])
        if month_idx < current_month:
            display_month = f"{cr.Fore.LIGHTBLACK_EX}{bulan}{cr.Fore.RESET}"
            display_payment = f"{cr.Fore.LIGHTBLACK_EX}Rp{payment:,}{cr.Fore.RESET}".replace(",", ".")
        else:
            display_month = bulan
            display_payment = f"Rp{payment:,}".replace(",", ".")

        monthly_data.append([display_month, display_payment])

    # hitung total
    total_bulanan = sum(v['jumlah'] for v in biaya_total.values() if v['bulanan'])
    total_sekali = sum(v['jumlah'] for v in biaya_total.values() if not v['bulanan'])

    total_df = pd.DataFrame({
        'Nama Tunggakan': ['Total Bulanan', 'Total Tahunan', 'Total Sekali Bayar'],
        'Tipe': ['Bulanan', 'Tahunan', 'Sekali'],
        'Jumlah': [
            f"Rp{total_bulanan:,}".replace(",", "."),
            f"Rp{total_bulanan * 12:,}".replace(",", "."),
            f"Rp{total_sekali:,}".replace(",", ".")
        ]
    })

    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))
    print(f"{cr.Fore.YELLOW}-> Hasil Perhitungan{cr.Fore.RESET}")
    print(tabulate(monthly_data, tablefmt='psql'))
    print(tabulate(total_df, headers='keys', tablefmt='psql', showindex=False))

    export_input = input("Apakah anda ingin mendownload tabel hasil tsb? (y/n): ").lower()
    if export_input == "y":
      clear_output()
      tanggal = datetime.now().strftime("%Y%m%d")
      export_as=f"data_biaya_{tanggal}"

      df_monthly = pd.DataFrame(monthly_data_clean, columns=["Bulan", "Pembayaran"])
      df_combined = pd.concat([
          df.rename(columns={"Nama Tunggakan": "Kategori"}),
          df_monthly,
          total_df.rename(columns={"Nama Tunggakan": "Kategori"})
      ], ignore_index=True).fillna("")
      print(tabulate(df_combined.fillna(""), headers='keys', tablefmt='psql', showindex=False))

      df_combined.to_csv(f"{export_as}.csv", index=False)
      with pd.ExcelWriter(f"{export_as}.xlsx", engine='openpyxl') as writer:
        df_combined.to_excel(writer, index=False, sheet_name='Data Biaya')

      with ZipFile(f"{export_as}.zip", "w") as z:
        z.write(f"{export_as}.csv")
        z.write(f"{export_as}.xlsx")

      characters = string.ascii_letters + string.digits
      clean_name = re.sub(r'\s+', '_', user_data['nama'])
      random_suffix = ''.join(random.choices(characters, k=5))
      bin_id = f"{clean_name}_{random_suffix}"

      with open(f"{export_as}.zip", "rb") as f:
        url = f"https://filebin.net/{bin_id}/{export_as}.zip"
        headers = {"Content-Type": "application/octet-stream"}
        response = req.post(url, headers=headers, data=f)
      if response.status_code == 201:
          print("✅ Upload berhasil!, download dibawah")
          print(f"🔗 Link file: https://filebin.net/{bin_id}/{export_as}.zip")
      else:
          print(f"❌ Upload gagal: {response.status_code}")
          print(response.text)

    input("\nTekan enter untuk kembali...")
    menu_awal()

# Mainmenu section -------------------------------------------------------------
appMaps = {
    f"👨‍⚕️{cr.Fore.BLUE}Dokterku {cr.Fore.LIGHTBLACK_EX}- Cek kesehatanmu disini!{cr.Fore.RESET}": dokterku,
    f"😜{cr.Fore.BLUE}Sahabat Belajarku {cr.Fore.LIGHTBLACK_EX}- Tetap nyaman meski sendiri disini!{cr.Fore.RESET}": sahabat_belajar,
    f"💸{cr.Fore.YELLOW}Kalkulator Pembayaran{cr.Fore.LIGHTBLACK_EX}- Bingung hitung bayaran? kesini!{cr.Fore.RESET}": kalkulator_pembayaran,
    f"❌{cr.Fore.RED}Keluar aplikasi{cr.Fore.RESET}" : keluar_program
}

pilihanTersedia = list(appMaps) # Convert dictionary to a list.

def listPilihan(pesan_kustom=None,list=appMaps,pesan_error=None):
    clear_output()
    if pesan_error:
      print(f"❗{cr.Fore.RED}{pesan_error}{cr.Fore.RESET}")
    print(f"{cr.Fore.BLUE}={cr.Fore.RESET}"*20)
    print(f"{cr.Back.LIGHTBLUE_EX}# SmartStudent App{cr.Back.RESET}\n -Solusi pintar untuk pelajar.")
    print(f"{cr.Fore.BLUE}={cr.Fore.RESET}"*20)
    if pesan_kustom is None and user_data["nama"] is not None:
      print(f"""{cr.Fore.GREEN}Selamat datang {cr.Fore.RESET+user_data["nama"]}👋, hari ini mood cek apanih?""")
    else:
       print(pesan_kustom)
    for i, name in enumerate(list.keys(), start=1):
        print(f"- {i}. {name}")

def menu_awal():
    listPilihan()
    while True:
        try:
            pilihan = int(input(f"Pilih aplikasi (1-{len(appMaps)}): "))
            if 1 <= pilihan <= (len(pilihanTersedia)):
              appMaps[pilihanTersedia[pilihan-1]]()
              break
            else:
              listPilihan(pesan_error="Pilihan tidak ada")
        except ValueError:
            listPilihan(pesan_error="Input tidak valid! Harus berupa angka sesuai list.")
        except KeyboardInterrupt:
            print("\nProgram dihentikan.")
            break
# Mainmenu section END----------------------------------------------------------

def inisialisasi():
    print(f"{cr.Back.BLUE}SELAMAT DATANG PELAJAR HEBAT{cr.Back.RESET}\n   /\t\t\t\ \nSilahkan perkenalkan diri anda")
    valid_nama = None
    while True:
      try:
        if user_data["nama"] is None:
          nama_input = input("Nama saya: ").strip()
          if len(nama_input) < 3 or any(char.isdigit() for char in nama_input):
              print(f"{cr.Fore.RED}Ayolah jangan bercanda, mana ada yang namanya kurang dari tiga huruf, spasi saja, atau angka.{cr.Fore.RESET}")
              continue
          user_data["nama"] = nama_input
        if user_data["kelamin"] is None:
            kelamin_input = input("Siswa/i?: ").strip().lower()
            if kelamin_input not in ["siswa", "siswi"]:
                print(f"{cr.Fore.RED}Ditanya Siswa atau Siswi jawabnya {kelamin_input}, ngaco kamu.{cr.Fore.RESET}")
                continue
            user_data["kelamin"] = (1 if kelamin_input == "siswa" else 0)
        if user_data["nama"] != None and user_data["kelamin"] != None:
            menu_awal()
            break
      except KeyboardInterrupt:
        print("Belum juga mulai udah keluar 😒")
      break

if __name__ == "__main__":
    inisialisasi()

# SmartStudent App
 -Solusi pintar untuk pelajar.
Selamat datang Juansyah Akbar👋, hari ini mood cek apanih?
- 1. 👨‍⚕️Dokterku - Cek kesehatanmu disini!
- 2. 😜Sahabat Belajarku - Tetap nyaman meski sendiri disini!
- 3. 💸Kalkulator Pembayaran- Bingung hitung bayaran? kesini!
- 4. ❌Keluar aplikasi

Program dihentikan.
